In [20]:
import pandas as pd
import numpy as np
import statistics
import requests
import json
import sys

In [21]:
def callApi(method, stock, isSeriese = False):
    key = "token=" + "pk_ee5af08dfb9a419aaba2cbee77b80165"
    host = "https://cloud.iexapis.com/stable/"
    ticker = "stock/"+ stock
    api_method ="/" + method 
    query = "?"
    add = "&"
    last5 = "/5y"
    last10 = "/10"
    period = "period=annual"
    if method == "dividends":
        url = host + ticker + api_method + last5 + query + key
    elif (method == "financials" or method == "balance-sheet") and isSeriese :
        url = host + ticker + api_method + last10 + query + period + add + key
    else:
        url = host + ticker + api_method + query + key 
    try:
        response = requests.request("GET", url)
        balance_sheet = json.loads(response.text)
        return balance_sheet
    except ValueError:
        print("Unknown expeption on" + stock)
        return {}

In [22]:
def avgDividends(dividend):
    avgDividend = np.array([])
    for dividend in dividends:
        if "amount" in dividend.keys() and dividend["amount"] != '': 
            if dividend["amount"] is not None:
                avgDividend = np.append(avgDividend, float(dividend["amount"]))
    if len(avgDividend) > 0: 
        return statistics.mean(avgDividend) 
    else:  
        return 0 

In [23]:
def sharesOutstanding(stats):
    if not("sharesOutstanding" in stats.keys()):
        return 0.0
    if stats["sharesOutstanding"] is not None:
        return float(stats["sharesOutstanding"])
    else:
        return 0.0

In [24]:
def futureValue(bookValue, period, rate ):
    return bookValue * ((1 + rate) ** period )

In [25]:
def bookValues(balancesheets, balancesheet):
    balancesheets["balancesheet"].insert(0, balancesheet["balancesheet"][0])
    if not("balancesheet" in balancesheets.keys()) or len(balancesheets["balancesheet"]) == 0:
        return []
    balancesheetsData = balancesheets["balancesheet"]
    bookValue = np.array([])
    for bs in balancesheetsData:
        treasuryStock = 0.0
        if bs["treasuryStock"] is not None:
            treasuryStock = bs["treasuryStock"]
           
        bookValue = np.append(bookValue, bs["shareholderEquity"]  / (bs["commonStock"] - treasuryStock))
    return bookValue


In [26]:
#COMPOUND ANNUAL GROWTH RATES (CAGR)
def CAGR(item):
    periods = len(item)
    if periods > 1 :
        first = item[-1]
        last = item[0] 
        cage = ((last/first)**(1/periods))-1
        return cage
    else:
        return 0

In [27]:
def BookValueGrowth_DividendValuation(balancesheets, lastBalancesheet, stats, dividends): 
    np.seterr(all='print')
    # Sharesoutstanding
    shares_outstanding = sharesOutstanding(stats)
    
    #Book Value
    bookValue = bookValues(balancesheets, lastBalancesheet) 
    
    # Current Year Book Value 
    currentYearBookValue = 0
    period = len(bookValue)
    if period >0:
        currentYearBookValue = bookValue[0]
    
    # Book value growth rate
    bookValueGrowthRate = CAGR(bookValue)
    adjustedbookValueGrowthRate = .1
    if bookValueGrowthRate > .05:
        adjustedbookValueGrowthRate =  bookValueGrowthRate
    
    # Future Book Value
    futureBookValue = futureValue(currentYearBookValue, len(bookValue), adjustedbookValueGrowthRate)
    print(futureBookValue)
    
    # Annual Dividend
    annualDividend = avgDividends(dividends)

    
    # Dicount Rate 10y fed note 
    dicountRate10YearFedNote = .0068
    #B14*((1-(1/(1+B15)^10))/B15)+(B21/(1+B15)^10))
    return annualDividend*((1-(1/(1+ dicountRate10YearFedNote)**period))/dicountRate10YearFedNote)+(futureBookValue/(1+dicountRate10YearFedNote)**period)
 

In [29]:
tickers = ['DAL']
dataSet = [] 
for ticker in tickers:
    data = {} 
    
    # API calls
    balancesheets = callApi("balance-sheet", ticker, True)
    balancesheet = callApi("balance-sheet", ticker)
    stats = callApi("stats", ticker)
    dividends = callApi("dividends", ticker)
    price = callApi("price", ticker)
    data["Ticker"] = ticker
    data["price"] = price
    
    #Book value growth and dividend valuation
    intrinsicValue =  BookValueGrowth_DividendValuation(balancesheets, balancesheet, stats, dividends)
    marginOfsafety = 1 - (float(price)/ intrinsicValue)
    annualReturn = CAGR([intrinsicValue, float(price)])
    data["Book Value Dividend"] =  {"IV": round(intrinsicValue, 2)},{"MS %": round(marginOfsafety *100,1) },{"AR %": round(annualReturn *100, 1)}
    dataSet.append(data.copy())
pd.DataFrame(dataSet) 

40.13081596631065


,Ticker,price,Book Value Dividend
0,DAL,31.7,"({'IV': 40.17}, {'MS %': 21.1}, {'AR %': 12.6})"


In [8]:
IntrinsicValue('AAPL')